# Digital Twin Fairways
WIP to use the OpenTNSim engine

In [1]:
import datetime
import json
import logging
import time

import dtv_backend.berthing

# the simpy processes and objects
import dtv_backend.compat

# library to load the fairway information network
import dtv_backend.fis
import dtv_backend.network
import dtv_backend.network.network_utilities
import dtv_backend.postprocessing
import dtv_backend.simple
import dtv_backend.simulate
import geojson
import networkx as nx
import pandas as pd
import shapely
import simpy
from networkx.readwrite import json_graph

# opentnsim import
from opentnsim import core

# reload for debugging purposes
%load_ext autoreload
%autoreload 2

logger = logging.getLogger("notebook")

/Users/baart_f/.virtualenvs/main/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning:

The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.



### Input
You can define your input in a json configuration file. The relevant parts are sites, fleet and climate.

In [2]:
# example input
with open("config.geojson") as f:
    config = geojson.load(f)

### Simulate with ``v3_run``

In [3]:
def v3_simulate(config):
    result = dtv_backend.simulate.v3_run(config)
    env = result['env']
    logbook = result['operator'].logbook
    response = {
        "log": logbook,
        "config": config,
        "env": {
            "epoch": env.epoch.timestamp(),
            "epoch_iso": env.epoch.isoformat(),
            "now": env.epoch,
            "now_iso": datetime.datetime.fromtimestamp(env.now).isoformat(),
        },
    }
    return response
    

In [4]:
resp = v3_simulate(config)

/Users/baart_f/src/digitaltwin-waterway/dtv_backend/dtv_backend/fis.py:111: ShapelyDeprecationWarning:

The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.



In [15]:
import numpy as np
import geopandas as gpd

result = dtv_backend.simulate.v3_run(config)

log_df = pd.DataFrame(result["operator"].logbook)
log_df["actor_name"] = log_df["Meta"].apply(lambda x: x["actor"].name)
log_df["actor_type"] = log_df["Meta"].apply(lambda x: type(x["actor"]).__name__)
log_df["state"] = log_df["Meta"].apply(lambda x: x["state"])
log_df["description"] = log_df["Meta"].apply(lambda x: x["description"])

# rename to these columns
columns = [
    "Start",
    "Stop",
    "Name",
    "Description",
    "Actor",
    "Actor type",
    "Geometry",
]
pivot_df = pd.DataFrame(
    # the pivot creates a multi-index, pick to create a single index
    log_df.pivot(index="ActivityID", columns="state")[
        [
            ("Timestamp", "START"),
            ("Timestamp", "STOP"),
            ("Message", "START"),
            ("description", "START"),
            ("actor_name", "START"),
            ("actor_type", "START"),
            ("geometry", "START"),
        ]
    ].values,
    columns=columns,
)

pivot_df["Start Timestamp"] = pivot_df["Start"].values.astype(np.int64) // 10**9
pivot_df["Stop Timestamp"] = pivot_df["Stop"].values.astype(np.int64) // 10**9

export_columns = [*columns, "Start Timestamp", "Stop Timestamp"]
pivot_df["Start"] = pivot_df["Start"].dt.strftime("%Y-%m-%d %H:%M:%S")
pivot_df["Stop"] = pivot_df["Stop"].dt.strftime("%Y-%m-%d %H:%M:%S")

# make geometry consistent
geometry = pivot_df["Geometry"].apply(
    lambda x: shapely.geometry.shape(x)
    if (hasattr(x, "geom_type") or isinstance(x, dict))
    else None
)
pivot_df['Geometry'] = geometry
pivot_df = gpd.GeoDataFrame(pivot_df[export_columns], geometry='Geometry')


In [16]:
json_str = pivot_df.to_json()

In [5]:
import pandas as pd
df = pd.DataFrame(resp['log'])
log_json = dtv_backend.postprocessing.log2json(df)
_ = json.dumps(log_json)


TypeError: Object of type LineString is not JSON serializable